In [1]:
rm -r progettoML
from IPython.display import clear_output
!pip3 install sentencepiece
!pip3 install transformers
!pip3 install pytorch_pretrained_bert
!pip install ekphrasis
!wget https://raw.githubusercontent.com/marcopoli/AlBERTo-it/master/vocab.txt
!git clone https://github.com/gioanat/progettoML
clear_output()

#!ln -ls progettoML/generate_utils.py generate_utils.oy # if import does not work
from progettoML.generate_utils import GenerateHints
from progettoML.generate_utils import parse_hashtags, AlBERTo_Preprocessing
obj = GenerateHints()
clear_output()

In [8]:
def run_server():
  import os
  os.system('wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip')
  os.system('unzip ngrok-stable-linux-amd64.zip')
  os.system('tensorboard --logdir /content/trainingdata/objectdetection/ckpt_output/trainingImatges/ --host 0.0.0.0 --port 6006 &')
  os.system('./ngrok http 6006 &')

  #!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  #!unzip ngrok-stable-linux-amd64.zip
  #get_ipython().system_raw('tensorboard --logdir /content/trainingdata/objectdetection/ckpt_output/trainingImatges/ --host 0.0.0.0 --port 6006 &')
  #get_ipython().system_raw('./ngrok http 6006 &')

  #!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"
  import subprocess
  output = None
  while output is None:
    try:
      output = subprocess.check_output('curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)[\'tunnels\'][0][\'public_url\'])"', shell=True)
      print(output.decode('utf-8'))
    except:
      print("Retrying..")

#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
print("Inserisci l'url nell'applicazione ed esegui la cella sottostante:")
run_server()

Retrying..
https://a77b8be1122e.ngrok.io



In [10]:
from flask import Flask, request, jsonify
from werkzeug.wrappers import Request, Response
app = Flask(__name__)

@app.route("/", methods=['GET'])
def health_check():
  return "OK"

@app.route("/lang", methods=['POST'])
def set_lang():
  data = request.get_json()['language']
  isItalian = data == 'it'
  print("Language set to", data, isItalian)
  set_language(isItalian)
  return "OK"

@app.route("/", methods=['POST'])
def hello():
  data = request.get_json()
  if data['whoami'] == 'Gioana':
      return "Ti amo tanto ❤️"
  return "Chi sei?"

@app.route("/generateTweets", methods=['POST'])
def generate_tweets():
  seed = request.get_json()['seed']
  ## Aggiungere parole
  phrases = obj.generate_phrase(seed, num_phrase = 3)
  return jsonify([parse_hashtags(phrase) for phrase in phrases])

@app.route("/generateWords", methods=['POST'])
def generate_response_hints():
  seed = request.get_json()['seed']
  hints = obj.generate_hints(seed, num_hints = 3)
  return jsonify(hints)

@app.route("/translate", methods=['POST'])
def translate():
  seed = request.get_json()['seed']
  a = AlBERTo_Preprocessing(do_lower_case=True)
  b = a.preprocess(seed)
  new_text = obj.traslated_funct(b)
  return parse_hashtags(new_text)

from werkzeug.serving import run_simple
run_simple('localhost', 6006, app)

 * Running on http://localhost:6006/ (Press CTRL+C to quit)
